# Tutorial 14: Params Classes

This tutorial walks you through the use of different parameters classes. Understanding how these classes work is fundamental in designing your own experiments in Flow. 

The classes featured in this tutorial are `InitialConfig`,`SumoCarFollowingParams`, `SumoParams` and `EnvParams`. Each of these classes is detailed in their corresponding sections in this tutorial.

This tutorial uses the classical Sugiyama experiment involving a loop (ring road) with a single lane and a fixed number of vehicles on it, similar to what is done in tutorial 1. This one though focuses on the use of the parameters in the classes.

**This tutorial is divided into 4 sections and is organized as follows:**

1. `InitialConfig`
2. `SumoCarFollowingParams`
3. `SumoParams`
4. `EnvParams`

## 1. InitialConfig

`InitialConfig` specifies parameters that affect the positioning of vehicles in the network at the start of a rollout. These parameters can be used for instance to limit the edges and number of lanes vehicles initially occupy, or to provide a means of adding randomness to the starting positions of vehicles. By default, the vehicles are uniformly distributed in the network, across all edges and lanes.

"the vehicles" means the vehicles that are initially present in the network when the simulation starts, i.e. not the vehicles added during the simulation via inflows but those added via `vehicles.add(num_vehicles=...)`.

### 1.1 List of possible parameters

Following are the parameters which can be set up within `InitialConfig`, along with their default values and description:


- `shuffle` (_bool, optional_): specifies whether the ordering of the vehicles should be shuffled. Default to False.
- `spacing` (_str, optional_): specifies the initial positioning of the vehicles. Can be either one of: 
    - "uniform" (default value): the vehicles will be placed uniformly
    - "random": the vehicles will be placed randomly
    - "custom": you must specify a custom ordering
- `min_gap` (_float, optional_): the minimum gap to be kept between two vehicles among all the vehicles, in meters. Defaults to 0 meters. 
- `x0` (_float, optional_): position of the first vehicle to be placed in the network. Can be used to shift the position of all the vehicles. The position is an edge position (c.f. the `specify_edge_starts` method in the scenario). Defaults to 0.
- `perturbation` (_float, optional_): standard deviation used to perturb the vehicles from their uniform position, in meters (this is only used in case `spacing` is set to "uniform"). Defaults to 0 meters.
- `bunching` (_float, optional_): reduce the portion of the network that should be filled with vehicles by this amount, in edge length. Defaults to 0. 
- `lanes_distribution` (_int, optional_): number of lanes the vehicles should be dispersed into. If the value is greater than the total number of lanes on an edge, vehicles are spread across all lanes. Defaults to infinity.
- `edges_distribution` (_str / str list / (str -> int) dict, optional_): edges vehicles may be placed on. Can be either one of:
    - "all": vehicles are distributed over all edges (default)
    - list of edges vehicles can be distributed over
    - dict where for each item, the key is the name of the edge and the value is the number of cars to place on that edge.
- `additional_params` (_dict, optional_): some other network-specific parameters.

### 1.2 Example using the Sugiyama experiment

The following lines of codes create a basic experiment in a ring, with 22 vehicles which initial position we can play with using `InitialConfig`. If you don't understand the following code, head over tutorial 1 which explains it.

Let's start with creating the vehicles that will be affected by the initial configuration, as well as the configuration itself:

In [5]:
from flow.core.params import VehicleParams, InitialConfig
from flow.controllers import IDMController, ContinuousRouter

# we add 22 vehicles that will be affected by the InitialConfig object
vehicles = VehicleParams()
vehicles.add("human",
        acceleration_controller=(IDMController, {}),
        routing_controller=(ContinuousRouter, {}),
        num_vehicles=22)

# set up initial config 
initial_config = InitialConfig(
    spacing="uniform",
    bunching=20
)

Then, let's write the rest of the code necessary for the simulation, and run it.

In [ ]:
from flow.core.experiment import Experiment
from flow.core.params import SumoParams, EnvParams, NetParams
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS
from flow.scenarios.loop import LoopScenario, ADDITIONAL_NET_PARAMS

# finish creating the experiment and run it
scenario = LoopScenario(
    name="sugiyama", 
    vehicles=vehicles, 
    net_params=NetParams(additional_params=ADDITIONAL_NET_PARAMS), 
    initial_config=initial_config)

env = AccelEnv(
    env_params=EnvParams(additional_params=ADDITIONAL_ENV_PARAMS), 
    sumo_params=SumoParams(render=True), 
    scenario=scenario
)

exp = Experiment(env)
exp.run(1, 1500)

TODO these actually don't work...

Feel free to play around with the parameters set in `InitialConfig`. For example, you can try with the followings:

In [11]:
initial_config = InitialConfig(
    spacing="uniform",
    perturbation=5
)

or, for instance,

In [9]:
initial_config = InitialConfig(
    spacing="random"
)

## 2. SumoCarFollowingParams

This section focuses on the discussion of parameters for SUMO-controlled acceleration behavior.
   
Following is the class declaration and the corresponding default values for the parameters:

   def __init__(
            self,
            speed_mode='right_of_way',
            accel=1.0,
            decel=1.5,
            sigma=0.5,
            tau=1.0,  # past 1 at sim_step=0.1 you no longer see waves
            min_gap=2.5,
            max_speed=30,
            speed_factor=1.0,
            speed_dev=0.1,
            impatience=0.5,
            car_follow_model="IDM",
            **kwargs):


### 2.1 List of possible parameters

The parameters within this class are:

* speed_mode = speed mode
* accel = acceleration (in m/s/s)
* decel = deceleration (in m/s/s)
* sigma = car-following parameter (visit the link below for details)
* tau = car-following parameter (visit the link below for details)
* min_gap = empty space after leader/spacing between vehicles (in meters)
* max_speed = vehicle's maximum velocity  (in m/s)
* speed_factor = the vehicles expected multiplicator for lane speed limits (visit the link below for details)
* speed_dev = the deviation of the speedFactor
* impatience = willingess of drivers to impede vehicles with higher priority (visit the link below for details)
* car_following_model = the car-following model to be used by the vehicles

More details available at: 
http://sumo.dlr.de/wiki/TraCI/Change_Vehicle_State#speed_mode_.280xb3.29

### 2.2 Example using the Sugiyama experiment

We again run the Sugiyama experiment, but now with focus on this class.

This part reinitializes the Sugiyama experiment. 

Similar to the first section of this tutorial, the following lines of codes call the necessary functions defined in Flow. This also adds vehicles, sets up the environment, and declares network parameters.

In [ ]:
from flow.controllers import IDMController, ContinuousRouter
from flow.core.experiment import Experiment
from flow.core.params import SumoParams, EnvParams, \
    SumoCarFollowingParams, NetParams, VehicleParams
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS
from flow.scenarios.loop import LoopScenario, ADDITIONAL_NET_PARAMS

name = "sugiyama_example"
sim_params = SumoParams(sim_step=0.1, render=True)

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params)
#initial_config = InitialConfig(bunching=20)

The SUMO Car Following Parameters class can be used within the declaration/addition of the vehicles to follow 

Look for `car_following_params=SumoCarFollowingParams (speed_mode="no_collide")` in the code below and that's where you can modify this within `SumoCarFollowingParams()` to make it work the way you like it. 

Run the following code to see for yourself that it is running until the cell where you should be able to run the simulation in the SUMO GUI.

In [ ]:
vehicles = VehicleParams()

vehicles.add("human",
        acceleration_controller=(IDMController, {}),
        routing_controller=(ContinuousRouter, {}),
        car_following_params=SumoCarFollowingParams
            (speed_mode="no_collide"),
        num_vehicles=22)

The cell below, when ran, completes the scenario and should be able to render the simulation in the SUMO GUI.

In [ ]:
# create the scenario object
scenario = LoopScenario(
    name="sugiyama",
    vehicles=vehicles,
    net_params=net_params)

# create the environment object
env = AccelEnv(env_params, sim_params, scenario)

exp = Experiment(env)

# run for a set number of rollouts / time steps
exp.run(1, 1500)

Observe the differences as you change or add the parameters declared within `SumoCarFollowingParams` in cell `In [5]`.

For example, instead of using `car_following_params=SumoCarFollowingParams(speed_mode="no_collide")`
     
try changing it to 
     
     `car_following_params=SumoCarFollowingParams(
            minGap=0.01,
            tau=0.5,
            speed_mode="aggressive"
            )`

Many things can be done within this class to conform to your needs, adjust this based on your preference/needs.

## 3. SumoParams

This is an extension of the class `SimParams` where it allows the users to have SUMO-specific simulation parameters.

These parameters are used to customize a SUMO simulation instance upon initialization. This includes passing the simulation step length, specifying whether to use SUMO's gui during a run, and other features described in the parameters below with their corresponding default values.
    
    def __init__(self,
                 port=None,
                 sim_step=0.1,
                 emission_path=None,
                 lateral_resolution=None,
                 no_step_log=True,
                 render=False,
                 save_render=False,
                 sight_radius=25,
                 show_radius=False,
                 pxpm=2,
                 overtake_right=False,
                 seed=None,
                 restart_instance=False,
                 print_warnings=True,
                 teleport_time=-1,
                 num_clients=1,
                 sumo_binary=None):

Below are the meaning and possible modifications of the parameters defined within this class:

       port: int, optional
            Port for Traci to connect to; finds an empty port by default
       sim_step: float optional
            seconds per simulation step; 0.1 by default
       emission_path: str, optional
            Path to the folder in which to create the emissions output. Emissions output is not generated if this value is not specified
        lateral_resolution: float, optional
            width of the divided sublanes within a lane, defaults to None (i.e. no sublanes). If this value is specified, the vehicle in the network cannot use the "LC2013" lane change model.
        no_step_log: bool, optional specifies whether to add sumo's step logs to the log file, and print them into the terminal during runtime, defaults to True
        render: str or bool, optional
            specifies whether to visualize the rollout(s)
            False: no rendering
            True: delegate rendering to SUMO-GUI for back-compatibility
            "gray": static grayscale rendering, which is good for training
            "dgray": dynamic grayscale rendering
            "rgb": static RGB rendering
            "drgb": dynamic RGB rendering, which is good for visualization
        save_render: bool, optional
            specifies whether to save rendering data to disk
        sight_radius: int, optional
            sets the radius of observation for RL vehicles (meter)
        show_radius: bool, optional
            specifies whether to render the radius of RL observation
        pxpm: int, optional
            specifies rendering resolution (pixel / meter)
        overtake_right: bool, optional
            whether vehicles are allowed to overtake on the right as well as the left
        seed: int, optional
            seed for sumo instance
        restart_instance: bool, optional
            specifies whether to restart a sumo instance upon reset. Restarting the instance helps avoid slowdowns cause by excessive inflows over large experiment runtimes, but also require the gui to be started after every reset if "render" is set to True.
        print_warnings: bool, optional
            If set to false, this will silence sumo warnings on the stdout
        teleport_time: int, optional
            If negative, vehicles don't teleport in gridlock. If positive,
            they teleport after teleport_time seconds
        num_clients: int, optional
            Number of clients that will connect to Traci


### Similar to the two previous sections, we'll use the Sugiyama experiment as an example.

In [ ]:
# This part initializes the Sugiyama experiment. 

from flow.controllers import IDMController, ContinuousRouter
from flow.core.experiment import Experiment
from flow.core.params import SumoParams, EnvParams, \
    InitialConfig, NetParams, VehicleParams
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS
from flow.scenarios.loop import LoopScenario, ADDITIONAL_NET_PARAMS
    
name = "sugiyama_example"

vehicles = VehicleParams()
    
vehicles.add("human",
        acceleration_controller=(IDMController, {}),
        routing_controller=(ContinuousRouter, {}),
        num_vehicles=22)
    

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params)
initial_config = InitialConfig()


This is the section where you should be able to modify the parameters within the `SumoParams` class.

In [ ]:
sim_params = SumoParams(sim_step=0.1, render=True)

# sim_step = 0.1 sets the simulation time step to 0.1
# render = True lets you see the simulation using the SUMO GUI

The cell below, when ran, completes the scenario and should be able to render the simulation in the SUMO GUI.

In [ ]:
# create the scenario object
scenario = LoopScenario(
    name="sugiyama",
    vehicles=vehicles,
    net_params=net_params)

# create the environment object
env = AccelEnv(env_params, sim_params, scenario)

exp = Experiment(env)

# run for a set number of rollouts / time steps
exp.run(1, 1500)

Observe the differences as you change or add the parameters declared within `SumoParams` in cell `In [8]`.

For example, instead of using 

`sim_params = SumoParams(sim_step=0.1, render=True)`
     
try changing it to 
     
     `sim_params = SumoParams(
            sim_step=0.1,
            render=True,
            emission_path="./data/"
            )`

There should now be emission files (CSV & XML files) created within the specified location (here, it should be at the `data` folder in Flow. The XML file is the ones that contains emission information based on the built-in SUMO emission model. If you did not add `emission_path="./data/`, no such files will be created.

You can add more parameters and specify the attributes within `SumoParams` to further modify the simulation.

## 4. EnvParams

This class enables you to specify the bounds of the action space and relevant coefficients to the reward function, as well as specifying how the positions of vehicles are modified in between rollouts.

Below is the definition of the class with the parameters and their corresponding default values:

    def __init__(self,
                 additional_params=None,
                 horizon=500,
                 warmup_steps=0,
                 sims_per_step=1,
                 evaluate=False):
                 
        self.additional_params = \
            additional_params if additional_params is not None else {}
        self.horizon = horizon
        self.warmup_steps = warmup_steps
        self.sims_per_step = sims_per_step
        self.evaluate = evaluate

        def get_additional_param(self, key):
        """Return a variable from additional_params."""
        return self.additional_params[key]


The parameters are detailed below and ideas on how to modify each one when used in a simulation.

            `additional_params`: dict, optional
                Specify additional environment params for a specific environment configuration
            `horizon`: int, optional
            `warmup_steps`: number of steps per rollouts (int, optional); this is the number of steps performed before the initialization of training during a rollout. These warmup steps are not added as steps into training, and the actions of RL agents during these steps are dictated by SUMO. Defaults to zero
           `sims_per_step`: int, optional number of SUMO simulation steps performed in any given rollout step. RL agents perform the same action for the duration of these simulation steps.
            `evaluate`: bool, optional
                flag indicating that the evaluation reward should be used so the evaluation reward should be used rather than the normal reward        

### Same to that of the three (3) previous sections, we'll use the Sugiyama experiment as an example.

In [ ]:
# This part initializes the Sugiyama experiment. 

from flow.controllers import IDMController, ContinuousRouter
from flow.core.experiment import Experiment
from flow.core.params import SumoParams, EnvParams, \
    InitialConfig, NetParams, VehicleParams
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS
from flow.scenarios.loop import LoopScenario, ADDITIONAL_NET_PARAMS

name = "sugiyama_example"

vehicles = VehicleParams()

vehicles.add("human",
        acceleration_controller=(IDMController, {}),
        routing_controller=(ContinuousRouter, {}),
        num_vehicles=22)

additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params)
initial_config = InitialConfig()

sim_params = SumoParams(sim_step=0.1, render=True)

### This is the section where you should be able to modify the parameters within the `EnvParams` class.

In [ ]:
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

### The cell below, when ran, completes the scenario and should be able to render the simulation in the SUMO GUI.

In [ ]:
# create the scenario object
scenario = LoopScenario(
    name="sugiyama",
    vehicles=vehicles,
    net_params=net_params)

# create the environment object
env = AccelEnv(env_params, sim_params, scenario)

exp = Experiment(env)

# run for a set number of rollouts / time steps
exp.run(1, 1500)

### Observe the differences as you change or add the parameters declared within `EnvParams` in cell `In [11]`.

For example, instead of using 

`env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)`
     
try changing it to: 
     
     `env_params = EnvParams(
            warmup_steps=40,
            sims_per_step=1,
            horizon=HORIZON,
            additional_params=additional_env_params,
            )`


You can add more parameters and specify the attributes within `EnvParams` to further modify the simulation.